<a href="https://colab.research.google.com/github/HuseyinK0r0glu/Machine_Learning/blob/main/PA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BBM 409 - Programming Assignment 2

**PART 1:** Binary Classification with SVM (30 points)  
**PART 2:** Multiclass Classification (70 points)


* You can add as many cells as you want in-between each question.
* Please add comments to your code to explain your work.  
* Please add Markdown cells to answer the (non-coding) questions in the homework text. You can, however, refer to the outputs of code cells without adding them as images to the Markdown cell unless you are requested to do otherwise.
* Please be careful about the order of runs of cells. Doing the homework, it is likely that you will be running the cells in different orders, however, they will be evaluated in the order they appear. Hence, please try running the cells in this order before submission to make sure they work.    
* Please refer to the homework text for any implementation detail. Though you are somewhat expected to abide by the comments in the below cells, they are mainly just provided for guidance. That is, as long as you are not completely off this structure and your work pattern is understandable and traceable, it is fine. For instance, you do not have to implement a particular function within a cell just because the comment directs you to do so.
* This document is also your report. Show your work.

###  Insert personal information (Hüseyin, Köroğlu, 2220356069)

In [ ]:
!pip install ucimlrepo

## Import Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree
from xgboost import XGBClassifier
from ucimlrepo import fetch_ucirepo
import warnings
warnings.filterwarnings('ignore')

---
# PART 1: BINARY CLASSIFICATION WITH SVM (30 POINTS)
---

## Load the Sonar Dataset from UCI Repository

In [ ]:
print("Loading Sonar dataset...")
sonar = fetch_ucirepo(id=151)
X_sonar = sonar.data.features
y_sonar = sonar.data.targets

print("X Values" , X_sonar)
print("Y Values" , y_sonar)

print("X Shape" , X_sonar.shape)
print("Y Shape" , y_sonar.shape)

# X.shape ===> (208,60) 208 samples and 60 attributes
# Y.shape ===> (208,1) consists of values 'R' for (Rock) and 'M' for (Mine)


Loading Sonar dataset...
X Values      Attribute1  Attribute2  Attribute3  Attribute4  Attribute5  Attribute6  \
0        0.0200      0.0371      0.0428      0.0207      0.0954      0.0986   
1        0.0453      0.0523      0.0843      0.0689      0.1183      0.2583   
2        0.0262      0.0582      0.1099      0.1083      0.0974      0.2280   
3        0.0100      0.0171      0.0623      0.0205      0.0205      0.0368   
4        0.0762      0.0666      0.0481      0.0394      0.0590      0.0649   
..          ...         ...         ...         ...         ...         ...   
203      0.0187      0.0346      0.0168      0.0177      0.0393      0.1630   
204      0.0323      0.0101      0.0298      0.0564      0.0760      0.0958   
205      0.0522      0.0437      0.0180      0.0292      0.0351      0.1171   
206      0.0303      0.0353      0.0490      0.0608      0.0167      0.1354   
207      0.0260      0.0363      0.0136      0.0272      0.0214      0.0338   

     Attribute7  

## Encode the Target Labels using LabelEncoder()

In [ ]:
encoder = LabelEncoder()
print("Before encoding")
print(y_sonar)
y_encoded = encoder.fit_transform(y_sonar.values.ravel())
print("After encoding")
print(y_encoded)

Before encoding
    class
0       R
1       R
2       R
3       R
4       R
..    ...
203     M
204     M
205     M
206     M
207     M

[208 rows x 1 columns]
After encoding
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


## Split the Data into 80% Training and 20% Testing Sets

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_sonar, y_encoded, test_size=0.2, random_state=42)

print("X_train shape" , x_train.shape)
print("X_test shape" , x_test.shape)
print("Y_train shape" , y_train.shape)
print("Y_test shape" , y_test.shape)

X_train shape (166, 60)
X_test shape (42, 60)
Y_train shape (166,)
Y_test shape (42,)


---
## 1.1. Linear Kernel SVM without Hyperparameter Tuning (5 points)
---

## Create a Pipeline with StandardScaler and Linear SVM

In [ ]:
# Here I specified the scaler so it will call the fit_transform() on training data and transform() on test data automatically
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(kernel='linear'))
])

## Train the Linear SVM Model

In [ ]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('svm', SVC(kernel='linear'))])

## Predict on the Test Set and Calculate Accuracies and Print it

In [ ]:
y_pred = pipeline.predict(x_test)

In [ ]:
train_accuracy_linear_svm = accuracy_score(y_train, pipeline.predict(x_train))
test_accuracy_linear_svm = accuracy_score(y_test, y_pred)

# Print results
print("Linear SVM Results")
print()
print("Training Accuracy: " , train_accuracy_linear_svm)
print("Test Accuracy: " , test_accuracy_linear_svm)

Linear SVM Results

Training Accuracy:  0.9578313253012049
Test Accuracy:  0.8333333333333334


## Display Classification Report for Linear SVM

In [ ]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Rock', 'Mine']))


Classification Report:
              precision    recall  f1-score   support

        Rock       0.95      0.77      0.85        26
        Mine       0.71      0.94      0.81        16

    accuracy                           0.83        42
   macro avg       0.83      0.85      0.83        42
weighted avg       0.86      0.83      0.84        42



## Display Confusion Matrix for Linear SVM

In [ ]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Confusion Matrix:
[[20  6]
 [ 1 15]]


---
## 1.2. SVM with GridSearchCV and 5-Fold Cross-Validation (15 points)
---

## Create a Pipeline for GridSearchCV

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

## Define the Parameter Grid for Different Kernels

In [ ]:
param_grid = {
    'svm__kernel' : ['linear' , 'rbf' , 'poly'],
    'svm__C': [0.1, 1, 10, 100],
    'svm__gamma': ['scale', 'auto']
}

## Create Stratified 5-Fold Cross-Validation

In [ ]:
cross_validation = StratifiedKFold(n_splits=5, shuffle=True, random_state=64)

## Initialize and Run GridSearchCV on Training Data

In [ ]:
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='accuracy',
    cv=cross_validation,
    verbose=1,
    n_jobs=-1
)

grid_search.fit(x_train, y_train)


Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=64, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('svm', SVC())]),
             n_jobs=-1,
             param_grid={'svm__C': [0.1, 1, 10, 100],
                         'svm__gamma': ['scale', 'auto'],
                         'svm__kernel': ['linear', 'rbf', 'poly']},
             scoring='accuracy', verbose=1)

## Display Best Parameters and Cross-Validation Score

In [ ]:
print("Best parameters" , grid_search.best_params_)
print("Best cross-validation score" , grid_search.best_score_)

Best parameters {'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}
Best cross-validation score 0.8677361853832443


## Evaluate the Best Model on the Test Set

In [ ]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)

train_accuracy = accuracy_score(y_train, best_model.predict(x_train))
test_accuracy = accuracy_score(y_test, y_pred)

print("Training Accuracy: " , train_accuracy)
print("Test Accuracy: " , test_accuracy)


Training Accuracy:  1.0
Test Accuracy:  0.9285714285714286


## Display Top 5 Parameter Combinations

In [ ]:
results = pd.DataFrame(grid_search.cv_results_)

# Select relevant columns and sort by mean test score
top5 = results.sort_values(by='mean_test_score', ascending=False).head(5)

# Display the result
print(top5[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']])

                                               params  mean_test_score  \
16  {'svm__C': 10, 'svm__gamma': 'auto', 'svm__ker...         0.867736   
19  {'svm__C': 100, 'svm__gamma': 'scale', 'svm__k...         0.867736   
22  {'svm__C': 100, 'svm__gamma': 'auto', 'svm__ke...         0.867736   
13  {'svm__C': 10, 'svm__gamma': 'scale', 'svm__ke...         0.867736   
7   {'svm__C': 1, 'svm__gamma': 'scale', 'svm__ker...         0.837433   

    std_test_score  rank_test_score  
16        0.055315                1  
19        0.055315                1  
22        0.055315                1  
13        0.055315                1  
7         0.059092                5  


## Display Classification Report for Tuned SVM

In [ ]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Rock', 'Mine']))


Classification Report:
              precision    recall  f1-score   support

        Rock       0.96      0.92      0.94        26
        Mine       0.88      0.94      0.91        16

    accuracy                           0.93        42
   macro avg       0.92      0.93      0.93        42
weighted avg       0.93      0.93      0.93        42



## Display Confusion Matrix for Tuned SVM

In [ ]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Confusion Matrix:
[[24  2]
 [ 1 15]]


## Compare Linear SVM vs Tuned SVM Results. Compare experimental results. Explain the performance impact of linear and nonlinear kernels. Why is kernel trick important? Also explain why using k-fold cross validation is more advantageous than train-test split. (10 points)

The Linear SVM uses a straight-line (linear) decision boundary, which works well when data is linearly separable but may perform poorly on complex datasets. The Tuned SVM with nonlinear kernels (like RBF or polynomial) can capture more complex relationships, improving accuracy by mapping data into a higher-dimensional space. This ability comes from the kernel trick, which allows SVMs to operate in high-dimensional spaces without explicitly computing the transformation, making it computationally efficient. K-fold cross-validation is more advantageous than a simple train-test split because it trains and tests the model multiple times on different data subsets, giving a more reliable and unbiased estimate of model performance while reducing the effect of random data division.

In [ ]:
print("\n=== Model Comparison ===")
print("Linear SVM - Training Accuracy " , train_accuracy_linear_svm)
print("Linear SVM - Test Accuracy " , test_accuracy_linear_svm)
print("Tuned SVM - Train Accuracy " , train_accuracy)
print("Tuned SVM - Test Accuracy" , test_accuracy)

# Determine which model performed better on the test set
if test_accuracy > test_accuracy_linear_svm:
    print("\nTuned SVM performed better on the test data.")
elif test_accuracy < test_accuracy_linear_svm:
    print("\nLinear SVM performed better on the test data.")
else:
    print("\nBoth models performed equally well on the test data.")


=== Model Comparison ===
Linear SVM - Training Accuracy  0.9578313253012049
Linear SVM - Test Accuracy  0.8333333333333334
Tuned SVM - Train Accuracy  1.0
Tuned SVM - Test Accuracy 0.9285714285714286

Tuned SVM performed better on the test data.


---
# PART 2: MULTICLASS CLASSIFICATION (70 POINTS)
---

## Load the Dry Beans Dataset from UCI Repository

In [ ]:
print("Loading Dry Beans dataset...")
dry_bean = fetch_ucirepo(id=602)
X_beans = dry_bean.data.features
y_beans = dry_bean.data.targets

print("X Values" , X_beans)
print("Y Values" , y_beans)

print("X Shape" , X_beans.shape)
print("Y Shape" , y_beans.shape)

distinct_classes = set()
for y in y_beans.squeeze():
    distinct_classes.add(y)
print("Distinct Classes" , distinct_classes)

for label, encoded_value in zip(encoder.classes_, range(len(encoder.classes_))):
    print(f"{label} → {encoded_value}")

# X.shape ===> (13612,16) 13612 samples and 16 attributes
# Y.shape ===> (13612,1)


Loading Dry Beans dataset...
X Values         Area  Perimeter  MajorAxisLength  MinorAxisLength  AspectRatio  \
0      28395    610.291       208.178117       173.888747     1.197191   
1      28734    638.018       200.524796       182.734419     1.097356   
2      29380    624.110       212.826130       175.931143     1.209713   
3      30008    645.884       210.557999       182.516516     1.153638   
4      30140    620.134       201.847882       190.279279     1.060798   
...      ...        ...              ...              ...          ...   
13606  42097    759.696       288.721612       185.944705     1.552728   
13607  42101    757.499       281.576392       190.713136     1.476439   
13608  42139    759.321       281.539928       191.187979     1.472582   
13609  42147    763.779       283.382636       190.275731     1.489326   
13610  42159    772.237       295.142741       182.204716     1.619841   

       Eccentricity  ConvexArea  EquivDiameter    Extent  Solidity  Round

## Encode the Target Labels using LabelEncoder

In [ ]:
encoder = LabelEncoder()
print("Before encoding")
print(y_beans)
y_encoded = encoder.fit_transform(y_beans.values.ravel())
print("After encoding")
print(y_encoded)

Before encoding
          Class
0         SEKER
1         SEKER
2         SEKER
3         SEKER
4         SEKER
...         ...
13606  DERMASON
13607  DERMASON
13608  DERMASON
13609  DERMASON
13610  DERMASON

[13611 rows x 1 columns]
After encoding
[5 5 5 ... 3 3 3]


## Split the Data into 80% Training and 20% Testing Sets

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X_beans , y_encoded , test_size=0.2 , random_state=64)

## Scale the Features using StandardScaler

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

---
## 2.1. Multinomial Logistic Regression (20 points)
---

## Define the Multinomial Logistic Regression Class

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
def one_hot_encoding(y, num_classes):
    result = np.zeros((len(y), num_classes))
    result[np.arange(len(y)), y] = 1
    return result

def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

class MultinomialLogisticRegression(BaseEstimator, ClassifierMixin):
  def __init__(self , learning_rate = 0.01 , iterations = 1000):
    self.learning_rate = learning_rate
    self.iterations = iterations
    self.weights = None
    self.bias = None

  def predict(self, X):
    logits = np.dot(X, self.weights) + self.bias
    probs = softmax(logits)
    return np.argmax(probs, axis=1)

  def fit(self , X , y):
    n_samples , n_features = X.shape
    n_classes = len(np.unique(y))
    y_one_hot = one_hot_encoding(y , n_classes)

    self.weights = np.zeros((n_features , n_classes))
    self.bias = np.zeros(n_classes)

    for i in range(self.iterations):

      linear_model = np.dot(X , self.weights) + self.bias

      probabilities = softmax(linear_model)

      dw = (1 / n_samples) * np.dot(X.T, (probabilities - y_one_hot))
      db = (1 / n_samples) * np.sum(probabilities - y_one_hot, axis=0)

      self.weights -= self.learning_rate * dw
      self.bias -= self.learning_rate * db

      if i % 100 == 0:
        loss = -np.mean(np.sum(y_one_hot * np.log(probabilities + 1e-8), axis=1))
        print(f"Iteration {i}: Loss = {loss:.4f}")

    return self



## Define Hyperparameter Grid for Multinomial Logistic Regression

In [ ]:
# param_grid = {
#     'learning_rate': [0.001, 0.01, 0.1],
#     'iterations': [500, 1000, 2000]
# }

# small one
param_grid = {
    'learning_rate': [0.01, 0.1],
    'iterations': [500, 1000]
}

## Run GridSearchCV for Multinomial Logistic Regression

In [ ]:
model = MultinomialLogisticRegression()

# could be use
# grid_search = GridSearchCV(
#     model,
#     param_grid=param_grid,
#     scoring='accuracy',
#     cv=3,
#     n_jobs=-1,
#     verbose=2
# )

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', verbose=2)
grid_search.fit(X_train, y_train)

print("\nBest Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

trainAccuracyValueMLR = grid_search.best_score_


Fitting 5 folds for each of 4 candidates, totalling 20 fits
Iteration 0: Loss = 1.9459
Iteration 100: Loss = 1.2213
Iteration 200: Loss = 0.9947
Iteration 300: Loss = 0.8750
Iteration 400: Loss = 0.7954
[CV] END .................iterations=500, learning_rate=0.01; total time=   5.9s
Iteration 0: Loss = 1.9459
Iteration 100: Loss = 1.2177
Iteration 200: Loss = 0.9902
Iteration 300: Loss = 0.8707
Iteration 400: Loss = 0.7915
[CV] END .................iterations=500, learning_rate=0.01; total time=   6.7s
Iteration 0: Loss = 1.9459
Iteration 100: Loss = 1.2158
Iteration 200: Loss = 0.9890
Iteration 300: Loss = 0.8698
Iteration 400: Loss = 0.7906
[CV] END .................iterations=500, learning_rate=0.01; total time=   5.1s
Iteration 0: Loss = 1.9459
Iteration 100: Loss = 1.2096
Iteration 200: Loss = 0.9821
Iteration 300: Loss = 0.8628
Iteration 400: Loss = 0.7836
[CV] END .................iterations=500, learning_rate=0.01; total time=   6.0s
Iteration 0: Loss = 1.9459
Iteration 100: Lo

## Evaluate Multinomial Logistic Regression on Test Set

In [ ]:
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

testAccuracyValueMLR = accuracy

Test Accuracy: 0.9258171134777818


## Display Classification Report for Multinomial Logistic Regression

In [ ]:
print("\nClassification Report:")
print("Classification Report:\n", classification_report(y_test, y_pred))


Classification Report:
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.90      0.93       263
           1       1.00      0.99      0.99        99
           2       0.91      0.95      0.93       308
           3       0.91      0.92      0.92       676
           4       0.96      0.94      0.95       394
           5       0.95      0.95      0.95       439
           6       0.87      0.89      0.88       544

    accuracy                           0.93      2723
   macro avg       0.94      0.93      0.94      2723
weighted avg       0.93      0.93      0.93      2723



## Display Confusion Matrix for Multinomial Logistic Regression

In [ ]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Confusion Matrix:
 [[236   0  14   0   2   1  10]
 [  0  98   1   0   0   0   0]
 [  5   0 294   0   3   1   5]
 [  0   0   0 622   1  14  39]
 [  0   0  12   4 371   0   7]
 [  2   0   0   8   0 415  14]
 [  0   0   1  46   8   4 485]]


---
## 2.2. Decision Tree (20 points)
---

###  Define the Decision Tree model. Use GridSearchCV to find the best hyperparameters. Perform multiclass classification on the Dry Beans dataset. Print best hyperparameters, classification report and confusion matrix. (15 points)

## Define Hyperparameter Grid for Decision Tree

In [ ]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}

## Run GridSearchCV for Decision Tree

In [ ]:
model = DecisionTreeClassifier(random_state=64)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', verbose=2)
grid_search.fit(X_train, y_train)
print("Best Hyperparameters:", grid_search.best_params_)
print("Best CV Accuracy:", grid_search.best_score_)

trainAccuracyValueDT = grid_search.best_score_

y_pred = grid_search.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

testAccuracyValueDT = accuracy


Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.3s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.3s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.3s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.3s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.3s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5; total time=   0.3s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5; total time=   0.3s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5; total time=   0.3s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5; total time=   0.3s
[CV] END criterion=gini, max_

## Display Classification Report for Decision Tree

In [ ]:
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))


Classification Report:

              precision    recall  f1-score   support

           0       0.90      0.88      0.89       263
           1       1.00      1.00      1.00        99
           2       0.91      0.91      0.91       308
           3       0.89      0.91      0.90       676
           4       0.96      0.93      0.94       394
           5       0.93      0.93      0.93       439
           6       0.86      0.86      0.86       544

    accuracy                           0.91      2723
   macro avg       0.92      0.92      0.92      2723
weighted avg       0.91      0.91      0.91      2723



## Display Confusion Matrix for Decision Tree

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", cm)


Confusion Matrix:
 [[232   0  16   0   3   7   5]
 [  0  99   0   0   0   0   0]
 [ 16   0 281   0   4   1   6]
 [  0   0   0 613   3  16  44]
 [  6   0   9   3 365   0  11]
 [  2   0   0  17   0 408  12]
 [  3   0   3  57   6   7 468]]


###  Describe how a decision tree builds its decision structure. (5 points)
* A decision tree builds its structure by starting with all the training data at the root and recursively splitting it based on the feature that best separates the target classes, using measures like information gain or Gini impurity. Each split creates branches leading to subsets of the data, and this process continues until a stopping condition is met, such as reaching a maximum depth or achieving pure nodes. The terminal nodes, or leaves, then make predictions based on the majority class (for classification) or average value (for regression).

---
## 2.3. XGBoost (20 points)
---

###  Define the XGBoost model. Use GridSearchCV to find the best hyperparameters. Perform multiclass classification on the Dry Beans dataset. Print best hyperparameters, classification report and confusion matrix(10 points)

## Define Hyperparameter Grid for XGBoost

In [ ]:
# param_grid = {
#     'n_estimators': [50, 100, 150],
#     'max_depth': [3, 5, 7],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'subsample': [0.7, 0.8, 1.0],
#     'colsample_bytree': [0.7, 0.8, 1.0],
#     'gamma': [0, 0.1, 0.2]
# }

# smaller
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5],
    'learning_rate': [0.1, 0.3],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.7, 0.8],
}

## Run GridSearchCV for XGBoost

In [ ]:
model = XGBClassifier(objective='multi:softmax', num_class=7, use_label_encoder=False, eval_metric='mlogloss')

grid_search = GridSearchCV(estimator=model,param_grid=param_grid,scoring='accuracy',cv=3,verbose=1,n_jobs=-1)

grid_search.fit(X_train, y_train)

print("Best Hyperparameters:", grid_search.best_params_)
print("Best Accuracy on Training CV:", grid_search.best_score_)

trainAccuracyValueXG = grid_search.best_score_


Fitting 3 folds for each of 32 candidates, totalling 96 fits
Best Hyperparameters: {'colsample_bytree': 0.8, 'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 50, 'subsample': 0.8}
Best Accuracy on Training CV: 0.9274423637158175


## Evaluate XGBoost on Test Set

In [ ]:
y_pred = grid_search.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Test Set Accuracy:", accuracy)

testAccuracyValueXG = accuracy

Test Set Accuracy: 0.9305912596401028


## Display Classification Report for XGBoost

In [ ]:
print("Classification Report:\n", classification_report(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.92      0.93       263
           1       1.00      1.00      1.00        99
           2       0.94      0.95      0.94       308
           3       0.91      0.93      0.92       676
           4       0.98      0.95      0.97       394
           5       0.95      0.95      0.95       439
           6       0.88      0.88      0.88       544

    accuracy                           0.93      2723
   macro avg       0.94      0.94      0.94      2723
weighted avg       0.93      0.93      0.93      2723



## Display Confusion Matrix for XGBoost

In [ ]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Confusion Matrix:
 [[241   0  11   0   2   3   6]
 [  0  99   0   0   0   0   0]
 [  9   0 292   0   2   2   3]
 [  0   0   0 629   1  11  35]
 [  2   0   4   1 376   0  11]
 [  1   0   0  10   0 419   9]
 [  1   0   3  52   4   6 478]]


### What are the revolutionary features of XGBoost compared to other tree-based models? (10 points)
* XGBoost is revolutionary because it combines gradient boosting with regularization, making it both powerful and resistant to overfitting. Unlike traditional tree-based models, it uses parallel and distributed computing, handling sparse data efficiently, and supports missing value imputation automatically. Its clever use of shrinkage (learning rate), column subsampling, and tree pruning improves accuracy and speed, making it much faster and more scalable than standard boosting methods while often achieving state-of-the-art performance in many machine learning tasks.

###  Compare the classification results of your Multinomial Logistic Regression, Decision Tree, and XGBoost models on the Dry Beans dataset. Discuss the comparison in terms of overall model performance, risk of overfitting, model complexity and the scenarios in which each model is most effective. (10 points)
* On the Dry Beans dataset, Multinomial Logistic Regression, Decision Tree, and XGBoost produced similar classification accuracy. Multinomial Logistic Regression is simple and interpretable, but it may struggle with complex non-linear relationships. Decision Trees can capture non-linear patterns and interactions, but they are prone to overfitting, especially with deep trees. XGBoost, on the other hand, is a powerful ensemble method that combines multiple trees with gradient boosting, providing high accuracy while controlling overfitting through regularization and subsampling. Despite similar performance on this dataset, Logistic Regression is best for interpretability and small datasets, Decision Trees are suitable for fast, rule-based decisions, and XGBoost is most effective for larger, complex datasets where maximizing accuracy is important.

In [ ]:
# COMPARE
print("Train Accuracy For Multinominal Logistic Regression: " , trainAccuracyValueMLR)
print("Test Accuracy For Multinominal Logistic Regression: " , testAccuracyValueMLR)
print("Train Accuracy For Decision Tree: " , trainAccuracyValueDT)
print("Test Accuracy For Decision Tree: " , testAccuracyValueDT)
print("Train Accuracy For XGBoost: " , trainAccuracyValueXG)
print("Test Accuracy For XGBoost: " , testAccuracyValueXG)

Train Accuracy For Multinominal Logistic Regression:  0.9186246310771304
Test Accuracy For Multinominal Logistic Regression:  0.9258171134777818
Train Accuracy For Decision Tree:  0.9075120014611391
Test Accuracy For Decision Tree:  0.9056188027910393
Train Accuracy For XGBoost:  0.9274423637158175
Test Accuracy For XGBoost:  0.9305912596401028
